In [578]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\use\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\use\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Menyiapkan semua library yang dibutuhkan

In [579]:
# import packages
import csv
import numpy as np
import pandas as pd
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, losses
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt

### Data Wrangling

#### Gathering Data

In [580]:
# import data
dataset1_df = pd.read_csv('dataset/dataset1.csv')
dataset1_df #show data below

,ingredient_name,rating,functions,link
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...
...,...,...,...,...
26082,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN
26083,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN
26084,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN
26085,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN


In [581]:
# import data
dataset2_df = pd.read_csv('dataset/dataset2.csv')
dataset2_df #show data below

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Evens Skin Tone, Dark Spot Fading",Antioxidant,A more stable derivative of pure vitamin C (as...,"Free Radical Biology and Medicine, September ...",INGREDIENT DICTIONARY/SEARCH RESULTS\n3-O Ethy...,"3-O ethyl ascorbic acid is a stable, water- an...","3-O ethyl ascorbic acid is a stable, water- an..."
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Soothing","Antioxidant, Plant Extracts",NaN,NaN,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcai\nRa...,Acai (pronounced “ah-sigh-ee”) is a small berr...,Acai (pronounced “ah-sigh-ee”) is a small berr...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,"Antioxidant, Plant Extracts",NaN,"Journal of AOAC International, September 2018...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcerola ...,Acerola fruit extract (also known as _Malpighi...,Acerola fruit extract (also known as _Malpighi...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...,Hydration,Antioxidant,NaN,Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl C...,NaN,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,Peptides,Lab-made anti-aging peptide that promotes firm...,"Pharmaceuticals, July 2021, pages 1–22;;Jour...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl D...,Acetyl dipeptide-1 cetyl ester is a lab-engine...,Acetyl dipeptide-1 cetyl ester is a lab-engine...
...,...,...,...,...,...,...,...,...,...,...,...
26069,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Skin Conditioning. We have not yet...
26070,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Antioxidant. We have not yet rated...
26071,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Humectant. We have not yet rated t...
26072,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Functions: Humectant, Skin Conditioning. We h..."


### Assesing Data

#### Menilai Tabel dataset1_df

In [582]:
dataset1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26087 entries, 0 to 26086
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26087 non-null  object
 1   rating           26087 non-null  object
 2   functions        26072 non-null  object
 3   link             2412 non-null   object
dtypes: object(4)
memory usage: 815.3+ KB


In [583]:
dataset1_df.isna().sum()

ingredient_name        0
rating                 0
functions             15
link               23675
dtype: int64

In [584]:
print("Jumlah Duplikasi: ", dataset1_df.duplicated().sum())

Jumlah Duplikasi:  0


In [585]:
dataset1_df.describe()

,ingredient_name,rating,functions,link
count,26087,26087,26072,2412
unique,26084,6,4055,2412
top,Rubus Idaeus (Raspberry) Fruit Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...
freq,2,23675,5642,1


#### Menilai Tabel dataset2_df

In [586]:
dataset2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26074 non-null  object
 1   rating           26074 non-null  object
 2   functions        26060 non-null  object
 3   link             2399 non-null   object
 4   benefits         1524 non-null   object
 5   categories       2350 non-null   object
 6   glance           942 non-null    object
 7   references       880 non-null    object
 8   all              2382 non-null   object
 9   description      2370 non-null   object
 10  combined_text    26073 non-null  object
dtypes: object(11)
memory usage: 2.2+ MB


In [587]:
dataset2_df.isna().sum()

ingredient_name        0
rating                 0
functions             14
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64

In [588]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


In [589]:
dataset2_df.describe()

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
count,26074,26074,26060,2399,1524,2350,942,880,2382,2370,26073
unique,26074,6,4044,2399,90,392,927,367,2382,2350,4245
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...,Hydration,"Antioxidant, Plant Extracts","Is a skin-restoring ingredient, meaning it is ...",Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nZingiber...,Used as an emollient and thickening agent in c...,Functions: Skin Conditioning. We have not yet...
freq,1,23675,5642,1,501,255,3,241,1,7,5642


### Cleaning Data

#### Cleaning Tabel dataset1_df

##### Duplicate Data

In [590]:
dataset1_df.duplicated().sum()

np.int64(0)

In [591]:
dataset1_df.drop_duplicates(inplace=True)

In [592]:
print("Jumlah duplikasi: ", dataset1_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [593]:
dataset1_df.isna().sum()

ingredient_name        0
rating                 0
functions             15
link               23675
dtype: int64

In [594]:
dataset1_df['functions'] = dataset1_df['functions'].fillna("Unknown")

In [595]:
dataset1_df['link'] = dataset1_df['link'].fillna("No Link")

In [596]:
print(dataset1_df.isna().sum())


ingredient_name    0
rating             0
functions          0
link               0
dtype: int64


#### Cleaning Tabel dataset2_df

##### Duplicate Data

In [597]:
dataset2_df.duplicated().sum()

np.int64(0)

In [598]:
dataset2_df.drop_duplicates(inplace=True)

In [599]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [600]:
dataset2_df.isna().sum()

ingredient_name        0
rating                 0
functions             14
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64

In [601]:
dataset2_df['functions'] = dataset2_df['functions'].fillna("Unknown")

In [602]:
dataset2_df['link'] = dataset2_df['link'].fillna("No Link")

In [603]:
print(dataset2_df.isna().sum())

ingredient_name        0
rating                 0
functions              0
link                   0
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64


##### Drop Columns

In [604]:
dataset1_df = dataset1_df.drop(columns='link')


In [605]:
dataset2_df = dataset2_df.drop(columns=['benefits', 'categories', 'glance', 'references', 'all', 'description', 'combined_text', 'link'])


In [606]:
print(dataset2_df.isna().sum())

ingredient_name    0
rating             0
functions          0
dtype: int64


In [607]:
dataset2_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
26069,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
26070,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
26071,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
26072,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


### Exploratory Data Analysis (EDA)

#### Explore Tabel dataset1_df

In [608]:
dataset1_df.sample(5)

,ingredient_name,rating,functions
2828,4-Butyl-5-Methylthiazole,NOT RATED,Functions: Fragrance. We have not yet rated th...
25926,Yucca Schidigera Fruit,NOT RATED,Functions: Skin Protecting. We have not yet ra...
24780,Tipa-Myristate,NOT RATED,Functions: Cleansing Agent. We have not yet ra...
17297,PEG-12 Methyl Ether Lauroxy PEG-5 Amidopropyl ...,NOT RATED,Functions: Hair Conditioning. We have not yet ...
15483,Musa Sapientum Peel Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...


In [609]:
dataset1_df.describe(include="all")

,ingredient_name,rating,functions
count,26087,26087,26087
unique,26084,6,4056
top,Rubus Idaeus (Raspberry) Fruit Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,2,23675,5642


In [610]:
dataset1_df.ingredient_name.is_unique

False

In [611]:
dataset1_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26082                        Zostera Marina Extract
26083       Zygophyllum Qatarense Leaf/Stem Extract
26084    Zygosaccharomyces Microellipsoides Ferment
26085                     Zymomonas Ferment Extract
26086                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26087, dtype: object>

In [612]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset1_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)


                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26079                          Zymomonas Ferment Extract   
26080                         Zymomonas Ferment Filtrate   
26081                                biosaccharide gum-1   
26082                                        pH Adjuster   
26083                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [613]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset1_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)


                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4056 rows x 1 columns]


#### Explore Tabel dataset2_df

In [614]:
dataset2_df.sample(5)

,ingredient_name,rating,functions
10257,Ethyl Polysilicate,NOT RATED,Functions: Film Forming. We have not yet rated...
14164,Linalylformate,NOT RATED,Functions: Fragrance. We have not yet rated th...
6041,Calcium Hydrolyzed Collagen,NOT RATED,"Functions: Hair Conditioning, Skin Conditionin..."
13484,Kojyl Carboxy Sh-Oligopeptide-75 Amide,NOT RATED,"Functions: Antioxidant, Skin Conditioning, Ski..."
2535,2-Amino-4-Hydroxyethylaminoanisole,NOT RATED,Functions: Hair Dyeing. We have not yet rated ...


In [615]:
dataset2_df.describe(include="all")

,ingredient_name,rating,functions
count,26074,26074,26074
unique,26074,6,4045
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,1,23675,5642


In [616]:
dataset2_df.ingredient_name.is_unique

True

In [617]:
dataset2_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26069                        Zostera Marina Extract
26070       Zygophyllum Qatarense Leaf/Stem Extract
26071    Zygosaccharomyces Microellipsoides Ferment
26072                     Zymomonas Ferment Extract
26073                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26074, dtype: object>

In [618]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset2_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)

                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26069                          Zymomonas Ferment Extract   
26070                         Zymomonas Ferment Filtrate   
26071                                biosaccharide gum-1   
26072                                        pH Adjuster   
26073                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [619]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset2_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)

                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4045 rows x 1 columns]


##### Menggabungkan dataset1 dan dataset2

In [620]:
datasetAll_df = pd.concat([dataset1_df, dataset2_df], axis=0, ignore_index=True)
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


#### Lowercasing, menghapus karakter khusus, dan tokenisasi


In [621]:
def clean_text(text):
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()
        # Menghapus karakter khusus
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Menghapus spasi berlebih
        text = re.sub(r'\s+', ' ', text).strip()
    return text

In [622]:
datasetAll_df['ingredient_name'] = datasetAll_df['ingredient_name'].apply(clean_text)
datasetAll_df['functions'] =  datasetAll_df['functions'].apply(clean_text)

In [623]:
sentences = []
labels = []
stopwords = [ "a", "an", "in", "is", "the", "and", "also", "there" ]

### Menghapus stopwords dan melakukan lemmatization

In [624]:
def preprocess_text(text):
    if isinstance(text, str):
        # Tokenisasi
        tokens = text.split()
        # Menghapus stopwords dan lemmatization
        processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(processed_tokens)
    return text

In [625]:
datasetAll_df['functions'] = datasetAll_df['functions'].apply(preprocess_text)

### Export Clean Dataset

In [626]:
# Menyimpan dataset ke file CSV
datasetAll_df.to_csv("Dataset clean/datasetAll.csv", index=False)


In [627]:
dataset1_df.to_csv("Dataset clean/dataset1_clean.csv", index=False)
dataset2_df.to_csv("Dataset clean/dataset2_clean.csv", index=False)

### Pre-processing

In [628]:
# import data
datasetAll_df = pd.read_csv('Dataset clean/datasetAll.csv')
datasetAll_df #show data below

,ingredient_name,rating,functions
0,o ethyl ascorbic acid,BEST,potent antioxidant highly stable form vitamin ...
1,acai,BEST,pronounced ahsighee small berry deep purple co...
2,acerola fruit extract,BEST,fruit extract potent source antioxidant hydrat...
3,acetyl carnitine hcl,BEST,function antioxidant
4,acetyl dipeptide cetyl ester,BEST,labmade skinfirming peptide formed reaction ac...
...,...,...,...
52156,zostera marina extract,NOT RATED,function skin conditioning yet rated ingredien...
52157,zygophyllum qatarense leafstem extract,NOT RATED,function antioxidant yet rated ingredient chan...
52158,zygosaccharomyces microellipsoides ferment,NOT RATED,function humectant yet rated ingredient chance...
52159,zymomonas ferment extract,NOT RATED,function humectant skin conditioning yet rated...


In [629]:
print(datasetAll_df['rating'].unique())

['BEST' 'GOOD' 'AVERAGE' 'BAD' 'WORST' 'NOT RATED']


In [630]:
# Mengonversi nilai non-numerik ke angka
rating_mapping = {
    'BEST': 6,
    'GOOD': 5,
    'AVERAGE': 4,
    'BAD': 3,
    'WORST':2,
    'NOT RATED':1
}

datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)

C:\Users\use\AppData\Local\Temp\ipykernel_2104\567315464.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)


In [631]:
# Hapus baris yang tidak sesuai dengan konteks
datasetAll_df = datasetAll_df[datasetAll_df['rating'].apply(lambda x: str(x).isnumeric() or x in rating_mapping.keys())]


In [632]:
datasetAll_df['rating'] = pd.to_numeric(datasetAll_df['rating'], errors='coerce')

In [633]:
# Encoding categorical data (jika ada kolom kategori)
from sklearn.preprocessing import LabelEncoder

# Misal kolom 'functions' merupakan data kategori
le = LabelEncoder()
datasetAll_df['functions'] = le.fit_transform(datasetAll_df['functions'])

# Normalisasi atau Standardisasi data numerik
scaler = StandardScaler()

# Misalkan kolom numerik adalah ['rating'] (pastikan kolom ini ada)
datasetAll_df[['rating']] = scaler.fit_transform(datasetAll_df[['rating']])

# Split dataset menjadi train dan test
X = datasetAll_df.drop(columns=['ingredient_name', 'functions'])  # Fitur (tanpa target)
y = datasetAll_df['functions']  # Target (misalnya 'functions')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Periksa ukuran data
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

Train size: (41728, 1)
Test size: (10433, 1)


In [634]:
datasetAll_df

,ingredient_name,rating,functions
0,o ethyl ascorbic acid,3.937776,3388
1,acai,3.937776,3427
2,acerola fruit extract,3.937776,799
3,acetyl carnitine hcl,3.937776,1168
4,acetyl dipeptide cetyl ester,3.937776,2847
...,...,...,...
52156,zostera marina extract,-0.302609,2631
52157,zygophyllum qatarense leafstem extract,-0.302609,1363
52158,zygosaccharomyces microellipsoides ferment,-0.302609,2466
52159,zymomonas ferment extract,-0.302609,2458


In [635]:
X_train.to_csv("Dataset clean/X_train.csv", index=False)
X_test.to_csv("Dataset clean/X_test.csv", index=False)
y_train.to_csv("Dataset clean/y_train.csv", index=False)
y_test.to_csv("Dataset clean/y_test.csv", index=False)

### Prepare the dataset for training

In [640]:
vocab_size = 15000
embedding_dim = 32
max_length = 200
trunc_type='post'
padding_type='post'
oov_token = "<OOV>"
training_portion = 0.8

In [644]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert labels to numpy arrays
train_labels = np.array(y_train)
test_labels = np.array(y_test)

In [645]:
with open("Dataset clean/datasetAll.csv", 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

In [646]:
file_path = 'Dataset clean/datasetAll.csv'


In [648]:
train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

In [649]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

In [652]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (83457,) + inhomogeneous part.